# **RAG Tabanlı Sağlık Asistanı Chatbotu** 🏥

Bu notebook, Türkçe sağlık makalelerinden oluşan bir veri setini kullanarak bir Retrieval-Augmented Generation (RAG) tabanlı chatbot oluşturur. Chatbot, kullanıcıların sağlıkla ilgili sorularını yanıtlamak için Google Generative AI modellerini ve Chroma vektör veritabanını kullanır. Kullanıcı arayüzü Gradio ile sağlanır.

## Colab Ortamına Hazırlık 🔨
Bu bölümde, gerekli kütüphaneleri yükleyeceğiz ve ortamı hazırlayacağız.

In [ ]:
# 🧩 Gerekli kütüphaneleri yükle
!pip install -q \
    google-generativeai \
    huggingface_hub \
    datasets \
    pandas \
    gradio \
    sentence-transformers \
    langchain \
    langchain-core \
    langchain-chroma \
    langchain_google_genai \
    langchain_text_splitters

# Gerekli tüm modülleri import et
import os
import pandas as pd
import gradio as gr
import google.generativeai as genai

# Hugging Face ve Veri Seti
from huggingface_hub import login
from datasets import load_dataset, Features, Value

# Colab
from google.colab import userdata
from google.colab import drive

# LangChain Metin İşleme
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LangChain Vektör Deposu ve Gömme (Embeddings)
from langchain.embeddings.base import Embeddings
from sentence_transformers import SentenceTransformer
from langchain_chroma import Chroma

# LangChain LLM ve RAG Zinciri
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

print("✅ Tüm kütüphaneler başarıyla yüklendi ve import edildi.")

✅ Tüm kütüphaneler başarıyla yüklendi ve import edildi.


## Ortam Değişkenlerini Yükleme ve API Yapılandırması

Bu bölümde, Google Generative AI ve Hugging Face API anahtarlarını yükleyeceğiz. Colab'de `.env` dosyası yerine, API anahtarlarını Colab'in 'Secrets' kısmına ekleyebilirsiniz (Sol panel > Secrets). Güvenlik için bu yöntemi kullanmanız önerilir.

In [ ]:
# Colab Secrets'tan API anahtarlarını al
from google.colab import userdata

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
HF_TOKEN = userdata.get('HF_TOKEN')

# Google Generative AI yapılandırması
genai.configure(api_key=GEMINI_API_KEY)

# Hugging Face'e giriş yap
try:
    login(token=HF_TOKEN)
    print("✅ Hugging Face login başarılı.")
except Exception as e:  # Genel hata yakalama
    print(f"❌ Giriş başarısız: {e}")

✅ Hugging Face login başarılı.


## Veri Setini Yükleme ve DataFrame Hazırlığı

Bu bölümde, Hugging Face üzerindeki `umutertugrul/turkish-hospital-medical-articles` veri setini yükleyeceğiz, DataFrame'e çevireceğiz ve metinleri birleştirerek `content` sütunu oluşturacağız.

In [ ]:
from datasets import load_dataset
import pandas as pd

print("📦 Sadece 'acibadem' veri seti yükleniyor...")

# Veri setini yükle (sadece acibadem parçasını)
data_split = load_dataset(
    "umutertugrul/turkish-hospital-medical-articles",
    split='acibadem'
)

# Veri setinin sütunlarını kontrol et
print("Veri seti sütunları:", data_split.column_names)

# DataFrame'e dönüştür
print("Veri DataFrame'e dönüştürülüyor...")
df = data_split.to_pandas()

# Varsayılan olarak 'title' ve 'text' sütunlarını kullan, 'headings' yoksa hata vermesin
available_columns = [col for col in ['title', 'text'] if col in df.columns]
if not available_columns:
    raise ValueError("Gerekli sütunlar ('title' veya 'text') veri setinde bulunamadı!")

# Content sütununu oluştur
df['content'] = df.apply(
    lambda row: f"{row['title'] if 'title' in row else ''}\n\n{row['text'] if 'text' in row else ''}",
    axis=1
)

print(f"✅ Toplam kayıt (sadece acibadem): {len(df)}")
print(df.head())

📦 Sadece 'acibadem' veri seti yükleniyor...


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 09eec7ab-3424-4acd-a20b-2da4af9dfef5)')' thrown while requesting HEAD https://huggingface.co/datasets/umutertugrul/turkish-hospital-medical-articles/resolve/main/README.md
Retrying in 1s [Retry 1/5].


Veri seti sütunları: ['url', 'title', 'text', 'publish_date', 'update_date', 'scrape_date']
Veri DataFrame'e dönüştürülüyor...
✅ Toplam kayıt (sadece acibadem): 6339
                                                 url  \
0  https://www.acibadem.com.tr/ilgi-alani/baso-ba...   
1  https://www.acibadem.com.tr/ilgi-alani/endosko...   
2  https://www.acibadem.com.tr/ilgi-alani/ablasyo...   
3  https://www.acibadem.com.tr/ilgi-alani/hemoglo...   
4  https://www.acibadem.com.tr/ilgi-alani/adenit-...   

                                               title  \
0  Baso (Bazofil) Nedir? Baso Yüksekliği ve Düşük...   
1  Endoskopik Sleeve Gastroplasti Nedir? Nasıl Uy...   
2   Ablasyon Nedir? Ablasyon Tedavisi Nasıl Yapılır?   
3  🩸 Hemoglobin (HGB) Nedir? Hemoglobin Yüksekliğ...   
4  Adenit Nedir? Adenit Belirtileri, Nedenleri ve...   

                                                text             publish_date  \
0  Bazofil(Baso), bağışıklık sisteminin alerjik t...  25 Aralık 2024 Çarşamba  

## Metin Parçalama (Chunking)

Bu bölümde, metinleri daha küçük parçalara (chunk) ayıracağız. Bu işlem, vektör veritabanında daha verimli arama yapılmasını sağlar.

In [ ]:
print("🔹 Metinler parçalara ayrılıyor...")

# Text splitter ayarları
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,      # Her parça yaklaşık 1000 karakter
    chunk_overlap=50,    # Parçalar arasında 100 karakter örtüşme
    separators=["\n\n", "\n", ".", " "]
)

# Content kolonunu kullanarak tüm metinleri chunk'lara ayır
chunks = []
for text in df['content'].tolist():
    chunks.extend(text_splitter.split_text(text))

print(f"✅ Toplam chunk sayısı: {len(chunks)}")

🔹 Metinler parçalara ayrılıyor...
✅ Toplam chunk sayısı: 38544


## Vektör Veritabanı Oluşturma

Bu bölümde, metin parçalarını Google Generative AI embeddings ile vektörleştireceğiz ve Chroma vektör veritabanında saklayacağız.

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # Drive'ı bağladığınızdan emin olun

# ... (SentenceTransformerEmbeddings sınıfınız burada) ...
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self.model = SentenceTransformer(model_name)
    def embed_documents(self, texts):
        return self.model.encode(texts, convert_to_tensor=False).tolist()
    def embed_query(self, text):
        return self.model.encode([text], convert_to_tensor=False)[0].tolist()

embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# --- KALICI KAYIT İÇİN DÜZELTME ---
persist_directory = "/content/drive/MyDrive/chroma_index_local"
collection_name = "turkish_hospital_articles"

# Mevcut veritabanını KONTROL ET (Artık Google Drive'da)
if os.path.exists(persist_directory):
    # Mevcut veritabanını YÜKLE
    print("✅ Mevcut Chroma vektör veritabanı GOOGLE DRIVE'DAN yüklendi.")
    db = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding_model,
        collection_name=collection_name
    )
    print(f"📦 Toplam vektör sayısı: {db._collection.count()}")
else:
    # Yeni veritabanı OLUŞTUR (Sadece ilk çalıştırmada)
    print("🔹 Veritabanı bulunamadı. GOOGLE DRIVE'A yeni veritabanı oluşturuluyor...")
    print("🔹 (Bu işlem chunk sayısına göre zaman alabilir)")

    # 'chunks' değişkeninin bu kod bloğundan ÖNCE tanımlandığından emin olun.
    db = Chroma.from_texts(
        texts=chunks,  # Parçalara ayrılmış metinler
        embedding=embedding_model,
        persist_directory=persist_directory,
        collection_name=collection_name
    )
    print("✅ Chroma vektör veritabanı oluşturuldu ve GOOGLE DRIVE'A kaydedildi.")
    print(f"📦 Toplam vektör sayısı: {db._collection.count()}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 32478f9c-a217-4b0b-b7c8-bde6804e17c6)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2/resolve/main/./sentence_bert_config.json
Retrying in 1s [Retry 1/5].


🔹 Veritabanı bulunamadı. GOOGLE DRIVE'A yeni veritabanı oluşturuluyor...
🔹 (Bu işlem chunk sayısına göre zaman alabilir)
✅ Chroma vektör veritabanı oluşturuldu ve GOOGLE DRIVE'A kaydedildi.
📦 Toplam vektör sayısı: 38544


## RAG Pipeline Kurulumu

Bu bölümde, Retrieval-Augmented Generation (RAG) zincirini oluşturacağız. Bu zincir, kullanıcı sorularını yanıtlamak için Chroma veritabanından ilgili metinleri çeker ve Gemini Flash modelini kullanır.

In [ ]:
# LLM (Gemini Flash)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.6,
    max_tokens=600,
    google_api_key=GEMINI_API_KEY
)

# Chroma retriever
retriever = db.as_retriever(search_kwargs={"k": 3})

# Prompt şablonu (Türkçe, sağlık odaklı)
system_prompt = (
    "Sen bir sağlık asistanısın. Kullanıcıdan gelen soruları, "
    "Chroma veritabanından getirilen ilgili metin parçalarına dayanarak yanıtla. "
    "Eğer soruyu yanıtlamak için yeterli bilgi yoksa, 'Bu konu hakkında yeterli bilgiye sahip değilim.' de. Daha fazla bilgi iste.\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# RAG chain
chain = create_retrieval_chain(
    retriever=retriever,
    combine_docs_chain=create_stuff_documents_chain(llm=llm, prompt=prompt)
)

#Gradio fonksiyonu
def answer_question(user_input, history):
    try:
        # Önceki konuşmaları birleştirip modele bağlam olarak gönder
        context = "\n".join([f"Kullanıcı: {q}\nAsistan: {a}" for q, a in history])
        full_input = f"{context}\nKullanıcı: {user_input}\nAsistan:"

        # Modeli çağır (örnek olarak chain.invoke)
        response = chain.invoke({"input": full_input})["answer"]

        # Yeni mesajı geçmişe ekle
        history = history + [[user_input, response]]
        return "", history  # giriş kutusunu temizle
    except Exception as e:
        return "", history + [[user_input, f"Bir hata oluştu: {e}"]]

## Gradio Arayüzü

Bu bölümde, kullanıcı dostu bir web arayüzü oluşturmak için Gradio'yu kullanacağız. Colab'de çalışması için `share=True` ile başlatacağız.

In [ ]:
# Gradio arayüzü
with gr.Blocks(theme="soft") as demo:
    gr.Markdown("## 🏥 Sağlık Asistanı Chatbot")
    gr.Markdown(
        "Sağlıkla ilgili sorularınızı yanıtlayan chatbot. "
        "Hastalık belirtileri, tedaviler ve daha fazlası hakkında bilgi alabilirsiniz."
    )

    chatbot = gr.Chatbot(label="💬 Sohbet Alanı")
    user_input = gr.Textbox(
        lines=2,
        placeholder="Sorunuzu buraya yazın ve Enter’a basın...",
        label="Soru",
    )
    submit_btn = gr.Button("Gönder")

    # Butonla gönderme
    submit_btn.click(
        fn=answer_question,
        inputs=[user_input, chatbot],
        outputs=[user_input, chatbot]
    )

    # Enter ile gönderme
    user_input.submit(
        fn=answer_question,
        inputs=[user_input, chatbot],
        outputs=[user_input, chatbot]
    )

# Uygulamayı başlat
if __name__ == "__main__":
    demo.launch()


/tmp/ipython-input-3375178813.py:9: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Chatbot Yanıtları")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5f8e7bc67ff2919e1a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
